# Custom GitOps

Instead of using the GitOps comes with the MLOps project template, here I implement a custom version of GitOps, where a git commit to code commit would automatically trigger a pipeline execution.

Reference:
- [aws-samples / mlops-amazon-sagemaker-devops-with-ml](https://github.com/aws-samples/mlops-amazon-sagemaker-devops-with-ml)
- [Complete CI/CD with AWS CodeCommit, AWS CodeBuild, AWS CodeDeploy, and AWS CodePipeline](https://aws.amazon.com/blogs/devops/complete-ci-cd-with-aws-codecommit-aws-codebuild-aws-codedeploy-and-aws-codepipeline/)
- [AWS MLOps Framework](https://aws.amazon.com/solutions/implementations/aws-mlops-framework)

## 1. Create a build project

A codebuild project updates or creates a pipeline everytime source code changes.

Build script is defined under the root directory of source code: `codebuild-buildspec.yml`

In [20]:
import boto3
import sagemaker
from botocore.exceptions import ClientError

codebuild = boto3.client('codebuild')

In [21]:
code_build_repository = 'https://git-codecommit.us-east-1.amazonaws.com/v1/repos/sagemaker-CKGQA-p-kiqtyrraeiec-modelbuild'
build_output_bucket = 'sm-nlp-data'
build_output_prefix = 'project-kgqa/outputs/'
code_build_project_name = 'project-kgqa'

In [22]:
role = sagemaker.session.get_execution_role()
role

'arn:aws:iam::093729152554:role/service-role/AWSNeptuneNotebookRole-NepTestRole'

In [23]:
iam = boto3.client('iam')
sm_products_use_role = iam.get_role(RoleName='AmazonSageMakerServiceCatalogProductsUseRole')
sm_products_use_role_arn = sm_products_use_role['Role']['Arn']

In [24]:
try:
    response = codebuild.create_project(
        name=code_build_project_name,
        source={
            'type': 'CODECOMMIT',
            'location': code_build_repository,
            'buildspec': 'codebuild-buildspec.yml', # an alternate buildspec file relative to the value of the built-in CODEBUILD_SRC_DIR environment variable
        },
        artifacts={
            'type': 'S3',
            'location': build_output_bucket,
            'path': build_output_prefix
        },
        environment={
            'type': 'LINUX_CONTAINER', # LINUX_GPU_CONTAINER
            'image': 'aws/codebuild/standard:4.0', # The image tag or image digest that identifies the Docker image to use for this build project
            'computeType': 'BUILD_GENERAL1_LARGE',
            'environmentVariables': [
                {
                    'name': 'test',
                    'value': 'test'
                }
            ],
        },
        logsConfig={
            'cloudWatchLogs': {
                'status': 'ENABLED'
            },
        },
        serviceRole=role
    )
    code_build_project = response['project']
except ClientError as e:
    if e.response["Error"]["Code"] == 'ResourceAlreadyExistsException':
        print(f"CodeBuild project with name {code_build_project_name} already exists")
    code_build_project = codebuild.batch_get_projects(names=[code_build_project_name])['projects'][0]
    
code_build_project

{'name': 'project-kgqa',
 'arn': 'arn:aws:codebuild:us-east-1:093729152554:project/project-kgqa',
 'source': {'type': 'CODECOMMIT',
  'location': 'https://git-codecommit.us-east-1.amazonaws.com/v1/repos/sagemaker-CKGQA-p-kiqtyrraeiec-modelbuild',
  'buildspec': 'codebuild-buildspec.yml',
  'insecureSsl': False},
 'artifacts': {'type': 'S3',
  'location': 'sm-nlp-data',
  'path': 'project-kgqa/outputs/',
  'namespaceType': 'NONE',
  'name': 'project-kgqa',
  'packaging': 'NONE',
  'encryptionDisabled': False},
 'cache': {'type': 'NO_CACHE'},
 'environment': {'type': 'LINUX_CONTAINER',
  'image': 'aws/codebuild/standard:4.0',
  'computeType': 'BUILD_GENERAL1_LARGE',
  'environmentVariables': [{'name': 'test',
    'value': 'test',
    'type': 'PLAINTEXT'}],
  'privilegedMode': False,
  'imagePullCredentialsType': 'CODEBUILD'},
 'serviceRole': 'arn:aws:iam::093729152554:role/service-role/AWSNeptuneNotebookRole-NepTestRole',
 'timeoutInMinutes': 60,
 'queuedTimeoutInMinutes': 480,
 'encrypt

In [26]:
code_build_project_arn = code_build_project['arn']
code_build_project_arn

'arn:aws:codebuild:us-east-1:093729152554:project/project-kgqa'

## 2. Create an Event that Triggers Codebuild with Codecommit

In [17]:
import boto3
import json

events = boto3.client('events')

In [32]:
code_build_repository_arn = 'arn:aws:codecommit:us-east-1:093729152554:sagemaker-CKGQA-p-kiqtyrraeiec-modelbuild'
code_build_repository_branch = 'main'
build_trigger_rule_name = 'ModelBuildOnCommit'
build_trigger_rule_description = 'Run both pipeline when new commit arrives'
code_build_project_target_id = 'codecommit-CKGQA-modelbuild'


In [20]:
codecommit_pattern = {
    "detail-type": ["Mode Build CodeCommit Repository State Change"],
    "resources": [code_build_repository_arn],
    "source": ["aws.codecommit"],
    "detail": {
        "referenceType": ["branch"],
        "referenceName": [code_build_repository_branch]
    }
}
codecommit_pattern_json = json.dumps(codecommit_pattern)

'{"detail-type": ["Mode Build CodeCommit Repository State Change"], "resources": ["arn:aws:codecommit:us-east-1:093729152554:sagemaker-CKGQA-p-kiqtyrraeiec-modelbuild"], "source": ["aws.codecommit"], "detail": {"referenceType": ["branch"], "referenceName": ["main"]}}'

In [33]:
response = events.put_rule(
    Name=build_trigger_rule_name,
    EventPattern=codecommit_pattern_json,
    State="ENABLED",
    Description=build_trigger_rule_description,
    EventBusName="default",
    Tags=[
        {
            'Key': 'event',
            'Value': 'model-build-commited'
        },
    ],
)
response

{'RuleArn': 'arn:aws:events:us-east-1:093729152554:rule/ModelBuildOnCommit',
 'ResponseMetadata': {'RequestId': '2edaf038-f8f7-429e-a72a-f16bcbe51b7b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2edaf038-f8f7-429e-a72a-f16bcbe51b7b',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '75',
   'date': 'Thu, 23 Sep 2021 09:05:22 GMT'},
  'RetryAttempts': 0}}

In [51]:
response = events.put_targets(
    Rule=build_trigger_rule_name,
    EventBusName='default',
    Targets=[
        {
            "Id": code_build_repository_target_id,
            "Arn": code_build_project_arn,
            "RoleArn": sm_products_use_role_arn
        }
    ]
)
response

{'FailedEntryCount': 0,
 'FailedEntries': [],
 'ResponseMetadata': {'RequestId': '119ce638-b740-4dcd-ac41-93b4716b563a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '119ce638-b740-4dcd-ac41-93b4716b563a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '41',
   'date': 'Thu, 23 Sep 2021 09:21:29 GMT'},
  'RetryAttempts': 0}}